In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_available

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!ls /home/medic/

ls: cannot access '/home/medic/': No such file or directory


In [5]:
text = open('idiot_dostoevsky.txt', 'rb').read().decode(encoding='utf-8')
print(text[:400])

Федор Михайлович Достоевский


ИДИОТ



ЧАСТЬ ПЕРВАЯ


I.


В конце ноября, в оттепель, часов в девять утра, поезд Петербургско-Варшавской железной дороги на всех парах подходил к Петербургу. Было так сыро и туманно, что насилу рассвело; в десяти шагах, вправо и влево от дороги, трудно было разглядеть хоть что-нибудь из окон вагона. Из пассажиров были и возвращавшиеся из-за границы; но более были 


In [6]:
# create vocab in chars of text
vocab = sorted(set(text))
len(vocab)

134

In [7]:
# create mapping a chars
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
len(text_as_int)

1308193

#### Создание модели и вспомогательных процедур

In [9]:
len_seq = 150
examples_per_epoch = len(text)//(len_seq + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Ф
е
д
о
р


In [10]:
sequences = char_dataset.batch(len_seq + 1, drop_remainder = True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Федор Михайлович Достоевский\n\n\nИДИОТ\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\nI.\n\n\nВ конце ноября, в оттепель, часов в девять утра, поезд Петербургско-Варшавской железной дор'
'оги на всех парах подходил к Петербургу. Было так сыро и туманно, что насилу рассвело; в десяти шагах, вправо и влево от дороги, трудно было разглядеть'
' хоть что-нибудь из окон вагона. Из пассажиров были и возвращавшиеся из-за границы; но более были наполнены отделения для третьего класса, и все людом '
'мелким и деловым, не из очень далека. Все, как водится, устали, у всех отяжелели за ночь глаза, все назяблись, все лица были бледножелтые, под цвет тум'
'ана.\n\nВ одном из вагонов третьего класса, с рассвета, очутились друг против друга, у самого окна, два пассажира, - оба люди молодые, оба почти налегке,'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_exmaple, target_example in dataset.take(1):
    print('Input data:', repr(''.join(idx2char[input_exmaple.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data: 'Федор Михайлович Достоевский\n\n\nИДИОТ\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\nI.\n\n\nВ конце ноября, в оттепель, часов в девять утра, поезд Петербургско-Варшавской железной до'
Target data: 'едор Михайлович Достоевский\n\n\nИДИОТ\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\nI.\n\n\nВ конце ноября, в оттепель, часов в девять утра, поезд Петербургско-Варшавской железной дор'


In [13]:
BATCH_SIZE = 128
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(128, 150), dtype=tf.int64, name=None), TensorSpec(shape=(128, 150), dtype=tf.int64, name=None))>

In [14]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024

model = tf.keras.Sequential(
         [
           tf.keras.layers.Embedding(vocab_size, embedding_dim),
           tf.keras.layers.LSTM(rnn_units, return_sequences = True),
           tf.keras.layers.Dense(vocab_size)
         ])

In [15]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(rnn_units, return_sequences = True, recurrent_initializer = 'glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units, return_sequences = True, recurrent_initializer = 'glorot_uniform')
        self.gru3 = tf.keras.layers.GRU(rnn_units, return_sequences = True, recurrent_initializer = 'glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
    
    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        x = (x + x1) / 2
        return self.fc(x)
    
model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        
            tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

            tf.keras.layers.LSTM(rnn_units,
                                return_sequences=True,
                                stateful=True,
                                recurrent_initializer='glorot_uniform'),

             tf.keras.layers.LSTM(rnn_units,
                                return_sequences=True,
                                stateful=True,
                                recurrent_initializer='glorot_uniform'),

            tf.keras.layers.LSTM(rnn_units,
                                return_sequences=True,
                                stateful=True,
                                recurrent_initializer='glorot_uniform'),
            tf.keras.layers.LSTM(rnn_units,
                                return_sequences=True,
                                stateful=True,
                                recurrent_initializer='glorot_uniform'),

            tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(vocab_size = len(vocab),
                        embedding_dim = embedding_dim,
                        rnn_units = rnn_units,
                        batch_size = BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 150, 134) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (128, None, 128)          17152     
                                                                 
 lstm_1 (LSTM)               (128, None, 1024)         4722688   
                                                                 
 lstm_2 (LSTM)               (128, None, 1024)         8392704   
                                                                 
 lstm_3 (LSTM)               (128, None, 1024)         8392704   
                                                                 
 lstm_4 (LSTM)               (128, None, 1024)         8392704   
                                                                 
 lstm_5 (LSTM)               (128, None, 1024)         8392704   
                                                                 
 dense_2 (Dense)             (128, None, 134)         

In [22]:
example_batch_predictions[3]

<tf.Tensor: shape=(150, 134), dtype=float32, numpy=
array([[ 1.2437961e-07, -2.0338791e-06,  7.2742409e-07, ...,
         2.2297712e-07, -3.3068554e-06,  1.7922165e-06],
       [ 2.2050453e-06, -4.8406068e-06,  2.1342273e-06, ...,
         4.1593998e-07, -1.1741408e-05,  6.5693571e-06],
       [ 8.7120125e-06, -5.9314921e-06,  6.2684931e-06, ...,
        -2.7037477e-06, -2.7608248e-05,  1.5119494e-05],
       ...,
       [ 5.5824043e-05, -1.2949415e-03, -4.6371302e-04, ...,
        -1.4466045e-03, -1.3623759e-03, -1.4874645e-03],
       [ 5.5543474e-05, -1.3303298e-03, -5.0278462e-04, ...,
        -1.3634748e-03, -1.3735878e-03, -1.4979258e-03],
       [ 5.6167271e-05, -1.3575029e-03, -5.4132967e-04, ...,
        -1.2872502e-03, -1.3770250e-03, -1.5069363e-03]], dtype=float32)>

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[3], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [24]:
# example what give a not trained model
print("Input: \n", repr("".join(idx2char[input_example_batch[3]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ункте-с, но до известной лишь степени и никак не далее обстоятельств, касавшихся собственно одного того пункта… Это я понимаю и нисколько не жалуюсь.\n'

Next Char Predictions: 
 'еБШzgbMЧешheЛТ"zлwтфхзblCШC"@У-биЯБьюОrЗjа0ВP1_gaЗf.6СТЭeШЮ6с!иЖ8Кгw\nдІ12зтCПLюЮРпОg*О2…LDГгJfАЧЯикТнІо…vЛr-;ЕTбCCмГВКОhСMX(БшоfiДrbЕќю\nРШУP-"Гl@-бvaЧ'


#### Приступ к тренировке модели

In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('Predictioin shape: ', example_batch_predictions.shape)
print('scalar_loss: ', example_batch_loss.numpy().mean())

Predictioin shape:  (128, 150, 134)
scalar_loss:  4.8979797


In [26]:
model.compile(optimizer='adam', loss=loss)

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [28]:
EPOCHS = 200
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
67/67 [==============================] - 45s 590ms/step - loss: 3.4163
Epoch 2/200
67/67 [==============================] - 41s 600ms/step - loss: 3.2680
Epoch 3/200
67/67 [==============================] - 42s 613ms/step - loss: 3.2685
Epoch 4/200
67/67 [==============================] - 47s 678ms/step - loss: 3.2685
Epoch 5/200
67/67 [==============================] - 42s 616ms/step - loss: 3.2684
Epoch 6/200
67/67 [==============================] - 44s 630ms/step - loss: 3.2684
Epoch 7/200
67/67 [==============================] - 43s 627ms/step - loss: 3.1746
Epoch 8/200
67/67 [==============================] - 47s 677ms/step - loss: 2.7279
Epoch 9/200
67/67 [==============================] - 43s 621ms/step - loss: 2.4801
Epoch 10/200
67/67 [==============================] - 42s 615ms/step - loss: 2.3160
Epoch 11/200
67/67 [==============================] - 43s 625ms/step - loss: 2.1195
Epoch 12/200
67/67 [==============================] - 46s 675ms/step - loss: 1.9171
E

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_198'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 128)            17152     
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           4722688   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_8 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_9 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_10 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 134)           

In [33]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [34]:
text_ = generate_text(model, start_string=u"Было время когда я ")
print(text_)

Было время когда я оберную нельзя будет срасновенный требо подле меня, Евгения Павлович: - кончал, у меня тогда делали? Не на что мамаша два целковых купленная, в виде корпусом, облокотился на стол руки, и все трое тоже пристально стали рассматривать князя. Из другой комнаты показался Коля после того, как скрылись последние действующих лиц нашего рассказа до того изменилось, что нам чрезвычайно трудно приступать к продолжению без особых объяснений. И однако мы чувствует тихо, как он не мог устроги по фамилии Аглае и не потеряла говорить, то как раб и наемщик, не иначе… я не сержусь, а грущу-с.

- Лукьян Тимофеич, помилуйся, сумасшедшая! Повсегодованием.

- Он, право, очень мил, - пробормотал опять старичок Ивану Петровичу.

- Я вошел сюда с мукой в сердце, - продолжал князь, все с каким-то вызовом обводя всех кругом глазами и как будто обращаясь ко всем безразлично. Князь заметил, что он весь семейный человек, видимо, противо, что вот этот кашель Фердыщенко; - дело ясное, дело само за 